# Exercise 1

## Creating Data and Backtesting a Trading Strategy

---

## Overview

Welcome to this Excercise. We are now going to put our skills to the test and create a dataset and prepare our first backtests. Proper data preparation is essential to conducting succesful backtest, and we want to make sure that we build a solid foundation and develop proper techniques to ensure that we can build our datasets easily at scale and develop proper data wrangling and preparation approaches. In particular, we are going to cover the  essential steps that are necessary to succesfully build trading  models beginning from creating datasets from publicly available stock price data to conducting exploratory data analysis to finally building and testing models. We are then quickly shifts gears and illustrate a few advanced investment ideas that can be used for further analysis.

---

## Introduction

We are going to start with downloading data using the yfinance library and then prepare this data by computing returns to conduct our exploratory analysis.

---

## Key Features

1) First use the yfinance library to download both data sets. You will be using the constituents of the S&P 500. 

2) We will then start to compute returns.



In [48]:
# Import Libraries
import yfinance as yf
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
OMP_NUM_THREADS=2
import matplotlib.pyplot as plt

In [2]:
# We can decide whther we want to download the data or use the saved csv version of it
DOWNLOAD_DATA_FROM_API = False 
# Set the min required num of obs per ticker to 100
MIN_REQUIRED_NUM_OBS_PER_TICKER=100

In [47]:
if DOWNLOAD_DATA_FROM_API == True:
    # Get the list of S&P 500 constituents
    sp500_tickers = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]['Symbol'].tolist()
    
    # Filter out Class B shares that have a '.B' in the ticker name
    sp500_tickers = [ticker for ticker in sp500_tickers if '.B' not in ticker]
    
    # Define the start and end dates for historical data
    start_date = '2000-01-01'
    end_date   = '2024-05-01'
    
    # Download historical prices for the list of ticker sp500_tickers
    historical_prices = yf.download(sp500_tickers, start=start_date, end=end_date)

    # Remove the MultiIndex and keep only the second level
    historical_prices.columns = historical_prices.columns.droplevel(0)
    
    # Filter and keep only columns where the first level of the MultiIndex is 'Adj Close'
    historical_prices  = historical_prices.loc[:, historical_prices.columns.get_level_values(0) == 'Adj Close']

    # Count non-missing values for each ticker
    ticker_counts = historical_prices.count()

    # Filter out tickers with fewer than n=MIN_REQUIRED_NUM_OBS_PER_TICKER=100 non-missing values
    valid_tickers = ticker_counts[ticker_counts >= MIN_REQUIRED_NUM_OBS_PER_TICKER].index
    
    # Filter the DataFrame based on valid tickers
    historical_prices = historical_prices[valid_tickers]
    

else:
    # Read the previously download data
    historical_prices = pd.read_csv('historical_prices.csv', index_col='Date', parse_dates=True)
    historical_prices.columns.name = 'Ticker'

In [4]:
# Print the first 5 rows
historical_prices.head()

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2000-01-03,43.613007,NaN,0.844981,NaN,NaN,8.992848,1.277778,NaN,16.274675,28.438286,...,NaN,11.505342,NaN,6.977997,18.328693,NaN,4.680301,NaN,25.027779,NaN
2000-01-04,40.281452,NaN,0.773741,NaN,NaN,8.735912,1.270833,NaN,14.909400,26.999619,...,NaN,11.073115,NaN,7.138673,17.977634,NaN,4.586222,NaN,24.666668,NaN
2000-01-05,37.782795,NaN,0.785063,NaN,NaN,8.719849,1.388889,NaN,15.204174,27.393782,...,NaN,11.659699,NaN,7.414120,18.957697,NaN,4.609740,NaN,25.138889,NaN
2000-01-06,36.344185,NaN,0.717125,NaN,NaN,9.024967,1.375000,NaN,15.328290,26.644875,...,NaN,12.205125,NaN,7.345260,19.937763,NaN,4.570544,NaN,23.777779,NaN
2000-01-07,39.372852,NaN,0.751094,NaN,NaN,9.121321,1.451389,NaN,16.072987,27.393782,...,NaN,11.803776,NaN,7.345260,19.879248,NaN,4.468626,NaN,23.513889,NaN


In [5]:
# Use the pandas info function to verify the data types of the dataframe column
historical_prices.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6120 entries, 2000-01-03 to 2024-04-30
Columns: 501 entries, A to ZTS
dtypes: float64(501)
memory usage: 23.4 MB


In [6]:
# Create a function called 'computingReturns' that takes close prices and a list of integers (momentums) as an inpute 
def computingReturns(close_prices, list_of_momentums): 
    # Initialize the forecast horizon
    forecast = 1
    # Compute forward returns by taking percentage change of close prices
    # and shifting by the forecast horizon
    f_returns = close_prices.pct_change(forecast)
    f_returns = f_returns.shift(-forecast)
    # Convert the result to a DataFrame
    f_returns = pd.DataFrame(f_returns.unstack())
    # Name the column based on the forecast horizon
    name = "F_" + str(forecast) + "_d_returns"
    f_returns.rename(columns={0: name}, inplace=True)
    # Reset the index to make Ticker and Date as regular columns
    f_returns.reset_index(inplace=True)
    # Rename the index column to 'Ticker'
    f_returns.rename(columns={'level_0': 'Ticker'}, inplace=True)
    # Initialize total_returns with forward returns
    total_returns = f_returns
    
    # Iterate over the list of momentum values
    for i in list_of_momentums:   
        # Compute returns for each momentum value
        feature = close_prices.pct_change(i)
        feature = pd.DataFrame(feature.unstack())
        # Name the column based on the momentum value
        name = str(i) + "_d_returns"        
        feature.reset_index(inplace=True)
        # Rename columns and reset index
        feature.rename(columns={0: name, 'level_0': 'Ticker'}, inplace=True)
        # Merge computed feature returns with total_returns based on Ticker and Date
        total_returns = pd.merge(total_returns, feature, left_on=['Ticker', 'Date'], right_on=['Ticker', 'Date'], how='left', suffixes=('_original', '_right'))
      
    # Drop rows with any NaN values
    total_returns.dropna(axis=0, how='any', inplace=True) 
    # Set Date and Ticker as the index
    total_returns.set_index(['Date', 'Ticker'], inplace=True)

    # Return the computed total returns DataFrame
    return total_returns


In [8]:
# Create a list of momentums for 1d and 2d returns
list_of_momentums = [1,2]
# Run the function computingReturns and save the output as total_data
total_data = computingReturns(historical_prices, list_of_momentums)
#Print the first 5 rows of total_data
total_data.head()

,,F_1_d_returns,1_d_returns,2_d_returns
Date,Ticker,,,
2000-01-05,A,-0.038076,-0.062030,-0.133681
2000-01-06,A,0.083333,-0.038076,-0.097744
2000-01-07,A,0.060577,0.083333,0.042084
2000-01-10,A,-0.013599,0.060577,0.148958
2000-01-11,A,-0.020221,-0.013599,0.046154


## How many tickers does our dataset have? 

In [129]:
total_data.index.get_level_values(1).nunique()

501

## How many uniquye trading days does our dataset have? 

In [9]:
total_data.index.get_level_values(0).nunique()

6117

## How many missing values do we have?

In [10]:
total_data.isna().mean()

F_1_d_returns    0.0
1_d_returns      0.0
2_d_returns      0.0
dtype: float64

## What is the CAGR and Sharpe of an equal weighted Portfolio?

In [11]:
# Compute the daily mean of all stocks. This will be our equal weighted benchmark
df_daily_mean  = pd.DataFrame(total_data.loc[:,'F_1_d_returns'].groupby(level='Date').mean())

# Convert daily returns to cumulative returns
df_cum_returns = pd.DataFrame((df_daily_mean[['F_1_d_returns']]+1).cumprod())

# Calculate the number of years in the dataset
years = len(df_daily_mean) / 252  # Assuming 252 trading days in a year

# Compute the Compound Annual Growth Rate (CAGR)
cagr = round((df_cum_returns['F_1_d_returns'].iloc[-1]**(1/years)-1)*100,2)

print(f'The CAGR is: {cagr}%')

# Compute the Sharpe Ratio by annualizing the daily mean and the daily std
df_daily_mean_mean  = df_daily_mean[['F_1_d_returns']].describe().iloc[1,:] * 252
df_daily_mean_std   = df_daily_mean[['F_1_d_returns']].describe().iloc[2,:] * pow(252,1/2)

# Compute the Sharpe Ratio and print it out
sharpe  = df_daily_mean_mean/df_daily_mean_std

print(f'Sharpe Ratio of Strategy: {round(sharpe.iloc[0],2)}')

The CAGR is: 17.24%
Sharpe Ratio of Strategy: 0.88


### Plot the cumulative Returns of the equal weighted portfolio

In [15]:
import hvplot.pandas  
df_cum_returns.hvplot()

:Curve   [Date]   (F_1_d_returns)

In [20]:
# Compute the returns per calendar year and save them in a dataframe calendar_returns_eq_wgt
ann_returns = (pd.DataFrame((df_daily_mean[['F_1_d_returns']]+1).groupby(df_daily_mean.index.get_level_values(0).year).cumprod())-1)*100
calendar_returns  = pd.DataFrame(ann_returns['F_1_d_returns'].groupby(ann_returns.index.get_level_values(0).year).last())
# Plot the  returns per calendar year
calendar_returns.hvplot.bar(rot=30,  legend='top_left')#.opts(multi_level=False) 

:Bars   [Date]   (F_1_d_returns)

## We store the information on all stocks for use as our benchmark

In [22]:
# Store df_cum_returns as df_cum_returns_bm
df_cum_returns_bm = df_cum_returns
df_cum_returns_bm.rename(columns={'F_1_d_returns':'SP500_eq_wgt'}, inplace=True)
# Store ann_returns as ann_returns_bm
ann_returns_bm = ann_returns
# Store ann_returns as ann_returns_bm
calendar_returns_bm  = calendar_returns
calendar_returns_bm.rename(columns={'F_1_d_returns':'SP500_eq_wgt'}, inplace=True)

### Create a function to calculate the RSI Indicator and add the RSI to the total_returns dataframe

In [26]:
# Create a function to calculate RSI
def calculate_rsi(data, window=14):
    delta = data.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

In [32]:
# Calculate RSI for each ticker separately and add to the DataFrame
total_data['RSI'] = total_data.groupby('Ticker')[['F_1_d_returns']].transform(calculate_rsi)
# Drop missing values
total_data.dropna(inplace=True)
total_data.head(5)


,,F_1_d_returns,1_d_returns,2_d_returns,RSI
Date,Ticker,,,,
2000-02-11,A,0.014096,-0.017115,-0.004950,50.487748
2000-02-14,A,0.071136,0.014096,-0.003260,55.571923
2000-02-15,A,-0.010687,0.071136,0.086235,49.006474
2000-02-16,A,0.197531,-0.010687,0.059689,62.217517
2000-02-17,A,-0.033505,0.197531,0.184733,49.713531


###  Build a trading strategy using RSI

In [35]:
# Define trading strategy based on RSI
def trading_strategy(rsi):
    if rsi < 30:
        return 1  # Go long
    elif rsi > 70:
        return -1  # Go short
    else:
        return 0

# Apply trading strategy to each RSI value
total_data['Position'] = total_data['RSI'].transform(trading_strategy)
# Create Returns for each Trade
total_data['Realized_Ret'] = total_data['F_1_d_returns'] *  total_data['Position'] 

In [36]:
# Compute the daily mean of our trading strategy
df_daily_mean  = pd.DataFrame(total_data.loc[:,'Realized_Ret'].groupby(level='Date').mean())

# Convert daily returns to cumulative return
df_cum_returns = pd.DataFrame((df_daily_mean[['Realized_Ret']]+1).cumprod())

# Calculate the number of years in the dataset
years = len(df_daily_mean) / 252  # Assuming 252 trading days in a year

# Compute the Compound Annual Growth Rate (CAGR)
cagr = round((df_cum_returns['Realized_Ret'].iloc[-1]**(1/years)-1)*100,2)

print(f'The CAGR is: {cagr}%')

# Compute the Sharpe Ratio by annualizing the daily mean and the daily std
df_daily_mean_mean  = df_daily_mean[['Realized_Ret']].describe().iloc[1,:] * 252
df_daily_mean_std   = df_daily_mean[['Realized_Ret']].describe().iloc[2,:] * pow(252,1/2)

sharpe  = df_daily_mean_mean/df_daily_mean_std

print(f'Sharpe Ratio of Strategy: {round(sharpe.iloc[0],2)}')

The CAGR is: -3.08%
Sharpe Ratio of Strategy: -6.36


##  Produce the Confusion Matrix for our Strategy

In [46]:
# Apply trading strategy to each RSI value
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Create the predicted and true values of F_1_d_returns as being larger or smaller than 0
total_data['y'] = np.where(total_data['F_1_d_returns']>0,1,-1)
total_data['y_pred'] = np.where(total_data['Position']>0,1,-1)

# Define actual and predicted signals (for demonstration purposes)
actual_signals    = total_data['y'].tolist()
predicted_signals = total_data['y_pred'].tolist()

# Compute confusion matrix
cm = confusion_matrix(actual_signals, predicted_signals)

print(classification_report(actual_signals, predicted_signals))

              precision    recall  f1-score   support

          -1       0.49      1.00      0.65   1312856
           1       0.10      0.00      0.00   1389462

    accuracy                           0.48   2702318
   macro avg       0.29      0.50      0.33   2702318
weighted avg       0.29      0.48      0.32   2702318

